# Canevas pour le TP Vélib

**Travailler sur des données réelles**

On n'a pas le temps de travailler sur des API réelles, on va sauter l'étape de récupération des données sur internet et aller directement à la phase d'analyse... Mais ces données n'en sont pas moins réelles et bruitées.

Ce notebook permet aussi de consolider les acquis sur `pandas`, `numpy` ainsi que sur les fonctions d'affichage

In [ ]:
# imports nécessaires dans le cadre du TP
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
try:
    import seaborn as sns
except:
    !pip install seaborn
    import seaborn as sns

## Import des données dans pandas


In [ ]:
# nom de fichier (chemin d'accès à personnaliser)
fnamejs = 'ressources/dataVelibJSON.txt'

In [ ]:
# lecture du fichier
f= open(fnamejs,'r') # ouverture en lecture
data = f.read() # récupération d'une chaine de caractères... Données vraiment très brutes !
print(data[:500]) # voir ces données brutes (les 500 premiers caractères)
f.close()

### Passage automatique à une table contenant une entrée par ligne

**input** = '[{"status": "OPEN", "contract_name": "Paris", "name": "", "bonus": "True", "bike_stands": 50, "number": 31705, "last_update": 1410616150000, "available_bike_stands": 48, "banking": "True", "available_bikes": 1, "address": "", "lat": 48.8645278209514, "lng": 2.416170724425901, "alt": 74.37134552001953}, {"status": "OPEN", ...

**output**=
 $$D =  \left[\begin{array}{cccc}
 ind & A & B & C \\1 & x_{1}& y_{1} &z_{1}\\ & \vdots & \\\ell&  x_{\ell}& y_{\ell} &z_{\ell}
 \\ & \vdots & \\ N&  x_{N}& y_{N} &z_{N}
 \end{array}
 \right]$$

In [ ]:
df = pd.read_json(data, orient='records')

Tester et comprendre le fonctionnement des méthodes suivantes

In [ ]:
print(df.head()) # entete + 5 lignes => comprendre les différentes variables aléatoires descriptives
#print(df.index)
df.describe()

In [ ]:
# df.sort_index(axis=1, ascending=False)
# methode de selection de donnée:
df['lng']                   # recupération d'une colonne
df.loc[:,['lat','lng']]     # recupération de deux colonnes

df[df['lat']>48.85]         # sélection des données
df['status'].value_counts() # récupération des valeurs possibles + comptage des occurences des valeurs

In [ ]:
# récupération des données au format numpy
df['alt'].to_numpy()

### <span style="color:red"> EXO Transformation des données</span>

Définir une nouvelle colonne correspondant aux arrondissements parisiens
1. Nouvelle colonne:
```df['arr'] = ...```
1. Analyse de la colonne `number`: les deux premiers chiffres donnent l'arrondissement
1. Donner le comptage des valeurs (nombre de stations par arrondissement)

L'étude de cette variable fait apparaitre des stations hors des arrondissements... Nous allons les supprimer

Cette opération est non-triviale: il faut sélectionner les stations $\le$ 20.... Mais aussi les stations avec un arrondissement $>0$!

Pour effectuer cette opération, le plus simple est d'utiliser `np.logical_and` qui agrège efficacement les booléens contenus dans deux vecteurs

In [ ]:
# pour travailler sur Paris uniquement:
dfp =    # faire la sélection 
df = dfp # pour travailler sur df pendant tout le TP... le df d'origine est ici perdu.

# fonctions d'affichage

Beaucoup de fonctions numpy/matplotlib/pandas sont compatibles entre elles. Démonstration ci-dessous:

In [ ]:
plt.close('all') # tout fermer
plt.figure() # nouvelle figure
plt.scatter(df['lat'],df['lng']) # scatter plot
plt.show() # affichage de la figure en cours

# note: on peut obtenir le même genre de résultats avec plot, mais c'est moins joli !

## Etude de l'altitude

1. Le code donne une couleur pour l'altitude des stations
1. <span style="color:red"> EXO</span> Modifier ce code (ou ouvrir une nouvelle boite) pour mettre en évidence l'arrondissement d'appartenance de la station
1. <span style="color:red"> EXO</span> Modifier ce code (ou ouvrir une nouvelle boite) pour mettre en évidence le taux de remplissage des stations


In [ ]:
fig, ax = plt.subplots()
coll = ax.scatter(df['lat'], df['lng'] , s=10, c=df['alt'], picker = 5)
plt.grid(True) # affichage de la grille
plt.colorbar(coll) # affichage de la légende


## Calcul d'aggrégat complexe

La fonction crosstab de pandas permet de calculer des choses sur des facteurs croisés: on obtient ainsi des tables de contingence ou des statistiques plus avancées. 
Nous allons nous intéresser à la distribution des emplacements par arrondissement. Cet exemple est abordable car les distributions de probabilité sont dicrètes.

https://pbpython.com/pandas-crosstab.html


In [ ]:
print('Combien de stations avec N places dans les différents arrondissement?')
plt.figure()
plt.imshow(pd.crosstab(df['arr'], df['bike_stands']))
plt.show()


# Exercices

Etudier la distribution de probabilité entre altitude et disponibilité. Cet exercice est un exercice à tiroirs... Comportant les étapes suivantes:

1. Discrétisation de l'altitude pour travailler plus facilement
1. Définition de la disponibilité en pourcentage (nb de dispo / nb de stands)
1. Discrétisation de la disponibilité
1. Calcul de la matrice de contingence (`crosstab`)
1. Normalisation pour estimer la distribution $P(Alt, Dispo)$
    * Quel problème se pose?
1. Calcul de la distribution conditionnelle $P(Dispo | Alt)$
1. Afficher les stations dans un repère altitude / disponibilité
    * afficher la covariance de ces deux variables aléatoires et le coefficient de corrélation: proposer une interprétation des résultats


### Rappels sur le calcul d'une distribution conditionnelle

Pour mieux distinguer les différents cas de figure de remplissage des stations, nous décidons de séparer les différentes classes d'altitude. Ainsi, nous allons caractériser la disponibilité des Vélibs dans différents univers.

* Calculer P_D_A = 
$$P(D | A) = \frac{P(D, A)}{P(A)}$$
    * identifer les dimensions de la matrice cible
    * vérifier que $\forall i,\ \sum_j P(D = d_j | A= a_i) = 1$ [sanity check]
* Calculer l'espérance de disponibilité sachant l'altitude E_D_A:
$$\forall i,\qquad E[D|A = a_i] = \sum_j d_j P(D = d_j | A= a_i)$$
Comment s'interprète cette espérance?


In [ ]:

# Paramétrisation de la discrétisation
nA = 15
nD = 12 # on ne prend pas les mêmes dimensions pour éviter de laisser trainer des bugs

# discrétisation de l'altitude
index = pd.cut(df['alt'], nA, labels=False)

# visualiser la nouvelle variable pour comprendre le processus... Puis en faire une nouvelle colonne dans les données


In [ ]:
# calcul de la distribution jointe 

P_A_D =

# calcul de la distribution conditionnelle

P_D_sA =

In [ ]:
plt.figure()
plt.imshow(P_A_D) # affichage de la distribution jointe
plt.colorbar()

In [ ]:
# note: il existe des fonctions avancées dans la toolbox de stats qui font tous ces calculs automatiquement

plt.figure()
# ['ind_alt'] => indice d'altitude discrétisée ['ind_pc'] => indice de pourcentage de disponibilité
sns.jointplot(df['ind_alt'],df['ind_pc'], kind="hex", color="k") 
# plt.savefig('sns_p_AD.pdf')

# avec la regression
plt.figure()
# tracé en continu... On ne visualise pas vraiment la distribution jointe
sns.jointplot(df['alt'],df['pc_available'], kind="reg", color="k")
# plt.savefig('sns_p_AD_reg.pdf')


# logique de filtrage sur les distributions continues
plt.figure()
# tracé en continu + lissage => on visualise bien la distribution jointe !
sns.jointplot(df['alt'],df['pc_available'], kind="kde", color="k")
# plt.savefig('sns_p_AD_smooth.pdf')

# Pour aller plus loin

### Indépendance

L'indépendance est un phénomène critique lors de l'implémentation des méthodes... Avec deux variables aléatoires, il suffit de tester:
$$ X \perp Y \iff \forall i,j p(X = x_{i}, Y=y_{j})  = p(X = x_{i})\times p( Y=y_{j})$$
... Ce qui n'est jamais vérifié exactement sur des données réelles.

**La bonne question est donc: suis-je assez proche d'un phénomène d'indépendance?**

#### taille des stations (S) VS arrondissements (Arr)

* Etude de corrélation sur la taille des stations par rapport aux arrondissements
    * tracé de la distribution jointe (sns.jointplot)
    * calcul du coefficient de corrélation

=> la faible valeur de coefficient de corrélation de corrélation nous donne un indice, mais nous nous rappelons que dans ce sens là, ce n'est pas une démonstration

* Calcul d'indépendance exact:
    * Discrétiser (ou plutot redistribuer) les tailles de stations sur 10 valeurs
    * Calcul de la jointe P_ArrS (cf P_AD)
        * Attention aux indices arr entre 1 et 20 => indices entre 0 et 19
    * Calcul des marginales P_Arr, P_S (trivial à partir de la loi jointe)
    * Calcul de PI_ArrS = P_Arr x P_S 
        * Implémentation du calcul par double boucle => trivial
        * calcul matriciel => non trivial (il faut dessiner les matrices sur une feuille de brouillon)
            * transformation des vecteurs en matrice + usage de dot <br>
        PI_ArrS = P_Arr.reshape(Narr, 1).dot(P_S.reshape(1,Ns))
        * Comparaison des valeurs PI_ArrS vs P_ArrS <br>
        'diff = ((PI_ArrS - P_ArrS)**2).sum()'
        => aucune chance d'arriver à 0...


* Application du test de $\chi^2$ = mesure d'une distance entre distribution
    * Lien wikipedia : [https://fr.wikipedia.org/wiki/Test_du_χ²]
    * Lien interne : [http://mapsi.lip6.fr/pmwiki.php?n=Cours.Semaine5]
    * Mesure de la distance entre deux distributions  $P_t$ (distribution théorique, issue des marginales dans notre exemple) et $P_o$ (distribution jointe) 
    $$D = \sum_{i}\sum_j N \frac{(P_t(i,j) - P_o(i,j))^2}{P_t(i,j)}$$
    La mesure dépend du nombre d'observation $N$
    * Chaque distribution est caractérisée par un nombre de degrés de libertés qui vaut ici: 
    $$DoF = (|Arr| - 1)(|S|-1) = 171$$
    * La distance limite, avec $\alpha$ de marge d'erreur, est donnée par :<br>
    import scipy.stats as stats<br>
    stats.chi2.ppf($\alpha$, DoF)
    * **Peut-on conclure que l'arrondissement est indépendant de la taille des stations?**

### Visualisation de données, réduction de dimensionnalité

Il serait intéressant de visualiser la population des stations en tenant compte de leur altitude, histoire de détecter les stations qui sont au bord des grandes montées.

Il s'agit donc de visualiser des données 3D en 2D, donc de réduire la dimensionnalité des données.

L'algorithme de l'état de l'art pour effectuer cette opération (délicate) est TSNE

Les opérations à mener sont, dans l'ordre:
1. Extraction des données au format numpy : pandas => numpy
1. Normalisation des données (elles sont trop *serrées* par défaut et l'algorithme ne marche pas bien)
    * **Seule chose restant à faire dans cet exercice**
    * Colonne par colonne:
        * Retirer la moyenne pour centrer la variable descriptive en 0
        * Diviser par l'écart-type pour obtenir un écart-type unitaire sur la variable
1. Invocation de TSNE pour passer de 3D à 2D
1. Récupération des indices d'arrondissements pour faire un affichage plus joli (et surtout plus intelligible)
1. Affichage

Comment interpréter ce que vous avez sous les yeux?

In [ ]:
import sklearn.manifold as visu # accès à TSNE

# pour afficher les différents arrondissements de différentes couleurs (et forme)
style = [(s,c) for s in "o^<*" for c in "byrmck" ]

# passage dans un univers numpy
X = df.loc[:,['lat','lng','alt']].as_matrix()
A = df['arr'].as_matrix()
# TODO : centrage des données en [0, 0, 0]
# TODO : ecart type unitaire sur chacune des trois variables

# réduction de la dimensionnalité
X2d = visu.TSNE().fit_transform(X)
plt.figure()
indexes = np.unique(A)
for i in range(len(indexes)): # affichage arrondissement par arrondissement
    ind = indexes[i]
    plt.scatter(X2d[A==ind,0],X2d[A==ind,1] , s=10,\
     marker=style[int(ind)%len(style)][0],c=style[int(ind)%len(style)][1])
plt.legend(indexes)
plt.savefig('stations_tsne_arr.pdf')